In [1]:
import numpy
import pathlib
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys, os


Using TensorFlow backend.


In [13]:
sps = set(sys.path)
sps.add('..')
sys.path = list(sps)
print(sys.path)

from core.util import data

['', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/termcolor-1.1.0-py3.6.egg', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/beautifulsoup4-4.6.3-py3.6.egg', '/Users/samuellevin/.ipython', '/Users/samuellevin/git/levinsamuel/boisbot', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/Werkzeug-0.14.1-py3.6.egg', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow-1.12.0rc2-py3.6-macosx-10.9-x86_64.egg', '..', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/selenium-3.141.0-py3.6.egg', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/protobuf-3.6.1-py3.6-macosx-10.9-x86_64.egg', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/Keras_Applications-1.0.6-py3.6.egg', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy-1.1.0-py3.6-macosx-10

In [3]:
with open('../data/5mintweets.txt') as f:
    tweetstr=f.read()

tweets=tweetstr.split('\n')
tweets[0:10]

['today is day one of the biggest project ive ever tried. it is called 17776 httpswww.sbnation.coma17776-football ',
 'congratulations to all red sox fans who are staying up past three in the morning terrified that your unbelievably successful team wont win every game. you are now an honorary fox news grandpa',
 'welcome to college football saturday',
 '.mlb 13th inning, suckers! im taking you rubes for at least four free innings of baseball. beware of jon the boring content bandit',
 'things ive done in red dead 2 so far_- fought bear_- murdered by alligator_- killed 10 lawmen after being falsely accused of murder_- called a guy a coward and he ran away_- sat by a lake, reflected_- lassoed a guy off his horse, hogtied him, put him back on his own horse, left',
 'red dead redemption 2 is a wonderful game. sending a fruit basket to the person who made it',
 'twitter offered me extended logoff time if i volunteered to show up at the mall in my twitter uniform and recruit new posters',
 '

In [4]:
# Count characters and create map to integers
chars=sorted(set(tweetstr))
# Number of distinct characters
chardict=len(chars)
charcount=len(tweetstr)
char_map={c:i for i,c in enumerate(chars)}

In [5]:
# Create sub sequences of a fixed length to feed to network
seq_length = 100
dataX = []
dataY = []
for i in range(0, charcount - seq_length):
    seq_in = tweetstr[i:i + seq_length]
    seq_out = tweetstr[i + seq_length]
    dataX.append([char_map[char] for char in seq_in])
    dataY.append(char_map[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  82090


In [6]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(chardict)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [7]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
print(y.shape[1])
model.compile(loss='categorical_crossentropy', optimizer='adam')

47


In [8]:
# define the checkpoint
pathlib.Path("../out/checkpoints").mkdir(exist_ok=True)
filepath="../out/checkpoints/weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [9]:
# Fit model
# model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

In [10]:
fls=os.listdir("../model")
print(fls)
print(type(fls[0]))
help(str.find)
help(str.replace)

['weights-20-2.4457%%jon_bois%%.hdf5']
<class 'str'>
Help on method_descriptor:

find(...)
    S.find(sub[, start[, end]]) -> int
    
    Return the lowest index in S where substring sub is found,
    such that sub is contained within S[start:end].  Optional
    arguments start and end are interpreted as in slice notation.
    
    Return -1 on failure.

Help on method_descriptor:

replace(...)
    S.replace(old, new[, count]) -> str
    
    Return a copy of S with all occurrences of substring
    old replaced by new.  If the optional argument count is
    given, only the first count occurrences are replaced.



In [14]:
numpy.random.randint(0, 1000)
print(type(X))
# print(X[12])
print(X[12].shape)
thrdX = numpy.reshape(X[12], (1,100,1))
print(thrdX.shape)


<class 'numpy.ndarray'>
(100, 1)
(1, 100, 1)
